In [2]:
import requests
from bs4 import BeautifulSoup
myBaseUrl = "https://www.amazon.com/Bionix-Safestraw-Drinking-Thickened-Liquids/product-reviews/B00L3D2MY0/ref=cm_cr_getr_d_paging_btm_prev_1?ie=UTF8&reviewerType=all_reviews&pageNumber="
start_urls = []
for i in range(1, 3):
    start_urls.append(myBaseUrl + str(i))
HEADERS = {
    'authority': 'www.amazon.com',
'method' : 'POST',
'path' : '/gp/redirection/singapore.html',
'scheme' : 'https',
'accept':'*/*',
'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36'
    
}
response = []
for i in start_urls:
    response.append(requests.get(i,headers=HEADERS))

['https://www.amazon.com/Bionix-Safestraw-Drinking-Thickened-Liquids/product-reviews/B00L3D2MY0/ref=cm_cr_getr_d_paging_btm_prev_1?ie=UTF8&reviewerType=all_reviews&pageNumber=1', 'https://www.amazon.com/Bionix-Safestraw-Drinking-Thickened-Liquids/product-reviews/B00L3D2MY0/ref=cm_cr_getr_d_paging_btm_prev_1?ie=UTF8&reviewerType=all_reviews&pageNumber=2']


In [3]:
print(response)

[<Response [200]>, <Response [200]>]


In [4]:
#collects soup data
soups = []
for i in response:
    soups.append(BeautifulSoup(i.content))
    
#collects and cleans review data
reviews = []
for soup in soups:
    review = soup.findAll("span",{'class':"a-size-base review-text review-text-content"})
    for i in review:
        reviews.append(i.text)
strippedreviews = []
for review in reviews:
    strippedreviews.append(review.strip())

In [5]:
avg_rating = 3.4 #change this average as necessary
stars = []
for soup in soups:
    star = soup.findAll("span",{'class':"a-icon-alt"})
    for i in star: 
        stars.append(i.text)
strippedstars = [] #this list still contains the stars from the avg. rating, top positive review, and top negative review
for star in stars:
    strippedstars.append(star.strip())
#code to remove excess stars scraped from html 
finalstars = []
indexofavg_rating = 0; 
for i in range(len(strippedstars)):
    if str(avg_rating) in strippedstars[i]:
        indexofavg_rating = i
    elif i != indexofavg_rating:
        if i != indexofavg_rating + 1:
            if i != indexofavg_rating + 2:
                finalstars.append(strippedstars[i])

In [6]:
#collects and cleans title data 
titles = []
for soup in soups:
    title = soup.findAll("a",{'class':"a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold"})
    for i in title:
        titles.append(i.text)
strippedtitles = []
for title in titles:
    strippedtitles.append(title.strip())


In [10]:
#creates a csv file with title and review data 
import pandas as pd    

df = pd.DataFrame(strippedtitles, columns=["title"])   
df.to_csv('soupsafestrawthin.csv', index=False)  #change name of csv file as needed 
read = pd.read_csv("soupsafestrawthin.csv")
read["review"] = strippedreviews
read.to_csv("soupsafestrawthin.csv", index=False)
read["star"] = finalstars
read.to_csv("soupsafestrawthin.csv", index=False)
df = pd.read_csv("soupsafestrawthin.csv")

In [11]:
#runs sentiment analysis on collected data in csv file
from textblob import TextBlob
pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity


df['polarity'] = df['review'].apply(pol)
df['subjectivity'] = df['review'].apply(sub)
df

,title,review,star,polarity,subjectivity
0,It is great for me overall!,"Really good for me, with difficulty swallowing...",5.0 out of 5 stars,0.750000,0.675000
1,Easy straws for the elderly,It doesn’t work for me I have no suction my th...,1.0 out of 5 stars,0.000000,0.000000
2,4-5 time use,They don’t last too long,3.0 out of 5 stars,-0.025000,0.233333
3,You have to push the straw in very hard!,It helped my sister drink water again after ha...,5.0 out of 5 stars,0.000000,1.000000
4,Difficult to use,Difficult to use.,2.0 out of 5 stars,-0.500000,1.000000
5,Did not work for us,This did not work for my Mom who had a stroke....,1.0 out of 5 stars,-0.200000,0.000000
6,Awesome product!,My student with autism often chickens when dri...,5.0 out of 5 stars,0.600000,0.662963
7,but the product is pretty good l,"Size not the size I expected, but the product ...",3.0 out of 5 stars,0.283333,0.666667
8,Too hard to use for elderly mom,Requires too much suction. Person who would ne...,2.0 out of 5 stars,0.180000,0.320000
9,very small amount of liquid dispenses,We were a little disappointed after trying the...,3.0 out of 5 stars,-0.171875,0.371875
